In [2]:
# -*- coding: utf-8 -*-
import re
import joblib
import pickle
from collections import defaultdict

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_1.csv')
df = pd.concat([train,test],sort=True) #合并train和test，并且用is_train进行标记
df.head()

,accessAgent,appProtocol,bytesIn,bytesOut,catOutcome,destAddress,destGeoAddress,destGeoCity,destGeoCountry,destGeoLatitude,...,responseHeader,srcAddress,srcGeoAddress,srcGeoCity,srcGeoLatitude,srcGeoLongitude,srcPort,startTime,transProtocol,txId
0,MicroMessenger Client,http,252,1269,FALL,*.215.158.*,NaN,深圳,中国,114.085947,...,"Connection:close,Content-Type:application/octe...",*.20.56.*,NaN,NaN,NaN,NaN,43052,*** 15:16:44,TCP,2
1,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,http,709,129,OK,*.50.2.*,NaN,NaN,局域网,NaN,...,"Server:nginx/1.16.*,***,Content-Type:applicati...",*.11.33.*,NaN,NaN,NaN,NaN,49769,*** 09:58:07,TCP,0
2,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,http,522,343,OK,*.71.3.*,NaN,杭州,中国,120.153576,...,"***,Content-Type:application/json;charset=UTF-...",*.20.56.*,NaN,NaN,NaN,NaN,28117,*** 09:03:59,TCP,6
3,netdisk;6.8.9.1;PC;PC-Windows;10.0.18362;Windo...,http,723,100,OK,*.200.143.*,NaN,西安,中国,108.948024,...,"***,Content-Type:application/octet-stream,Conn...",*.11.33.*,NaN,NaN,NaN,NaN,61812,*** 17:08:59,TCP,0
4,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,http,723,297,OK,*.228.67.*,NaN,无锡,中国,120.301663,...,"Server:openresty,Connection:keep-alive,***,Con...",*.20.14.*,NaN,NaN,NaN,NaN,54467,*** 16:06:45,TCP,0


In [4]:
df.shape
train.head()

,eventId,startTime,transProtocol,appProtocol,name,srcAddress,srcPort,destAddress,destPort,srcGeoCity,...,requestUrl,httpReferer,accessAgent,responseCode,requestBody,requestHeader,responseHeader,requestContentType,responseContentType,label
0,292670958385078277,*** 15:16:44,TCP,http,HTTP请求访问,*.20.56.*,43052,*.215.158.*,80,NaN,...,"/hdip/static/isee.js/<img/src=""//xss.bxss.me/t...",NaN,MicroMessenger Client,400.0,NaN,"Accept:*/*,Cache-Control:no-cache,Connection:c...","Connection:close,Content-Type:application/octe...",application/octet-stream,application/octet-stream,0
1,292676887412228098,*** 09:58:07,TCP,http,HTTP请求访问,*.11.33.*,49769,*.50.2.*,80,NaN,...,/statistics,http://*.50.2.*/wiki/pages/viewpage.action?pag...,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,204.0,NaN,"Host:*.50.2.*,Connection:keep-alive,Accept:app...","Server:nginx/1.16.*,***,Content-Type:applicati...",NaN,application/json,1
2,291939933730643973,*** 09:03:59,TCP,http,HTTP请求访问,*.20.56.*,28117,*.71.3.*,80,NaN,...,/api/admin/public/gateInfo,http://admin.antivirus-ncov.dev.wochanye.com/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4...,200.0,NaN,"Host:admin.antivirus-ncov.dev.wochanye.com,Use...","***,Content-Type:application/json;charset=UTF-...",application/json;charset=UTF-8,application/json;charset=UTF-8,0
3,292673842473820162,*** 17:08:59,TCP,http,HTTP请求访问,*.11.33.*,61812,*.200.143.*,80,NaN,...,/statistics,NaN,netdisk;6.8.9.1;PC;PC-Windows;10.0.18362;Windo...,200.0,\r\n----iphone_BOUNDARY--\r\nContent-Dispositi...,"Host:xacu02.baidupcs.com,Range:bytes=198574080...","***,Content-Type:application/octet-stream,Conn...",NaN,application/octet-stream,1
4,288571275298418693,*** 16:06:45,TCP,http,HTTP请求访问,*.20.14.*,54467,*.228.67.*,80,NaN,...,/jsonapi,http://mini.flash.2144.com/20190806/mini-main-...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,200.0,...............c.>.f7..'.\r\nOy.....+f..?p.8.c...,"Accept:*/*,Referer:http://mini.flash.2144.com/...","Server:openresty,Connection:keep-alive,***,Con...",NaN,image/gif,1


In [5]:
train_path = 'train.csv'
df = pd.read_csv(train_path,encoding='utf-8')
labels = df["label"].values
df.shape

(10396, 37)

In [6]:
#map()批量转换数据类型
#
srcAddress_ip = df['srcAddress'].apply(lambda x: list(map(float,re.findall(r"\d+\.?\d*",x)))[0]).values
destAddress_ip = df['destAddress'].apply(lambda x: list(map(float,re.findall(r"\d+\.?\d*",x)))[0]).values
requestUrlQuery_len = df['requestUrlQuery'].apply(lambda x: len(x)).values
requestHeader_len = df['requestHeader'].astype(str).apply(lambda x: len(x)).values
responseHeader_len = df['responseHeader'].astype(str).apply(lambda x: len(x)).values
requestUrl_len = df['requestUrl'].apply(lambda x: len(x)).values
httpReferer_len = df['httpReferer'].astype(str).apply(lambda x: len(x)).values
requestBody_len = df['requestBody'].astype(str).apply(lambda x: len(x)).values

In [7]:
#选取该列数据，numerical特征
train_cat_outcome_onehot_encode = pd.get_dummies(df["catOutcome"]).values
train_numerical_features = df.loc[:, ["txId", "bytesOut", "bytesIn", "destGeoLatitude", 
                                      "destGeoLongitude", "srcPort", "destPort"]].values#选取该列数据，numerical特征


In [8]:
#缺失值太多的列
request_body_0 = df["requestBody"].isna().values.astype(int)#缺失值显示1
request_body_1 = df["requestBody"].notnull().values.astype(int)#非缺失值显示1

http_referer_0 = df["httpReferer"].isna().values.astype(int)
http_referer_1 = df["httpReferer"].notnull().values.astype(int)

request_content_type_0 = df["requestContentType"].isna().values.astype(int)
request_content_type__1 = df["requestContentType"].notnull().values.astype(int)

In [9]:
#计算requestUrlQuery的/ = &
request_url_query_slash_count = df["requestUrlQuery"].apply(lambda x: x.count("/")).values
request_url_query_equal_count = df["requestUrlQuery"].apply(lambda x: x.count("=")).values
request_url_query_and_count = df["requestUrlQuery"].apply(lambda x: x.count("&")).values

In [10]:
#计算requestUrl的. /
request_url_dot_count = df["requestUrl"].apply(lambda x: x.count(".")).values
request_url_slash_count = df["requestUrl"].apply(lambda x: x.count("/")).values

In [11]:
#取startTime的时，丢掉分
#x.split(":")[0][-2:]第一个冒号前面的两个字符
start_time_hour = df["startTime"].apply(lambda x: int(x.split(":")[0][-2:])).values

In [12]:
#计算responseHeader的，：-
response_header_comma_count = df["responseHeader"].astype(str).apply(lambda x: x.count(",")).values
response_header_colon_comma_count = df["responseHeader"].astype(str).apply(lambda x: x.count(":")).values
response_header_strigula_comma_count = df["responseHeader"].astype(str).apply(lambda x: x.count("-")).values

In [13]:
#计算requestHeader的,:;=./
request_header_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count(",")).values
request_header_colon_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count(":")).values
request_header_sem_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count(";")).values
request_header_equal_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count("=")).values
request_header_dot_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count(".")).values
request_header_slash_comma_count = df["requestHeader"].astype(str).apply(lambda x: x.count("/")).values

In [14]:
src_address_dict_map = defaultdict(int)
src_address = set(df["srcAddress"].values)#无序不重复元素集
#对于srcAddress进行label编码，生成字典并保存为pkl文件
for k, v in zip(src_address, range(1, len(src_address) + 1)):
    src_address_dict_map[k] = v
with open("src_address_dict_map.pkl", "wb") as fp:
    pickle.dump(src_address_dict_map, fp)

In [15]:
#判断上（0）下（1）午
def flitter0(x):
    num = int(x.split(":")[0][-2:])
    return 0 if 0 <= num <= 12 else 1
def flitter1(x):
    num = int(x.split(":")[0][-2:])
    return 1 if 12 <= num <= 23 else 0
start_time_0 = df["startTime"].apply(flitter0).values
start_time_1 = df["startTime"].apply(flitter1).values

In [16]:
src_address_map = df["srcAddress"].map(src_address_dict_map).values

In [28]:
train_features = np.hstack([srcAddress_ip.reshape(-1, 1),
                            destAddress_ip.reshape(-1, 1),
                            requestUrlQuery_len.reshape(-1, 1),
                            requestHeader_len.reshape(-1, 1),
                            responseHeader_len.reshape(-1, 1),
                            requestUrl_len.reshape(-1, 1),
                            httpReferer_len.reshape(-1, 1),
                            requestBody_len.reshape(-1, 1),
                            request_body_0.reshape(-1, 1),
                            request_body_1.reshape(-1, 1),
                            http_referer_0.reshape(-1, 1),
                            http_referer_1.reshape(-1, 1),
                            request_content_type_0.reshape(-1, 1),
                            request_content_type__1.reshape(-1, 1),
                            request_url_query_slash_count.reshape(-1, 1),
                            request_url_query_equal_count.reshape(-1, 1),
                            request_url_query_and_count.reshape(-1, 1),
                            request_url_dot_count.reshape(-1, 1),
                            request_url_slash_count.reshape(-1, 1),
                            start_time_hour.reshape(-1, 1),
                            response_header_comma_count.reshape(-1, 1),
                            response_header_colon_comma_count.reshape(-1, 1),
                            response_header_strigula_comma_count.reshape(-1, 1),
                            request_header_comma_count.reshape(-1, 1),
                            request_header_colon_comma_count.reshape(-1, 1),
                            request_header_sem_comma_count.reshape(-1, 1),
                            request_header_equal_comma_count.reshape(-1, 1),
                            request_header_dot_comma_count.reshape(-1, 1),
                            request_header_slash_comma_count.reshape(-1, 1),
                            src_address_map.reshape(-1, 1),
                            start_time_0.reshape(-1, 1),
                            start_time_1.reshape(-1, 1),
                            train_cat_outcome_onehot_encode, 
                            train_numerical_features])

In [29]:
train_features.shape

(10396, 41)

In [30]:
#特征筛选：待使用方法：F-test,递归循环使用
#train_features = train_features[:, [0, 29, 32, 33, 36, 15, 19, 35, 16, 39, 26, 
#                                    7, 2, 34, 14, 25, 18, 28, 21, 17]]

x_train, x_test, y_train, y_test = train_test_split(train_features,labels, test_size=0.25, random_state=2020)

In [31]:
train_features.shape

(10396, 41)

In [20]:
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [32]:
batch_size = 64

In [46]:
#定义神经网络一层
model = Sequential()
model.add(Dense(1024,input_dim=20,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=batch_size)
#loss,acc=model.evaluate(x_test,y_test)
y_train1 = y_test
y_pred = model.predict_classes(x_test)
accuracy = accuracy_score(y_train, y_pred)
#recall = recall_score(y_train1, y_pred , average="binary")
#precision = precision_score(y_train1, y_pred , average="binary")
f1 = f1_score(y_train, y_pred, average="binary")

122/122 [==============================] - 0s 2ms/step - loss: 372.9297 - accuracy: 0.5455
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


ValueError: Found input variables with inconsistent numbers of samples: [7797, 2599]

In [38]:
#定义神经网络两层
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=batch_size)
loss,acc=model.evaluate(x_test,y_test)

82/82 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.5310


In [41]:
#四层
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=batch_size)
loss,acc=model.evaluate(x_test,y_test)

82/82 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.5310


In [42]:
#五层
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(x_train,y_train,batch_size=batch_size)
loss,acc=model.evaluate(x_test,y_test)

82/82 [==============================] - 0s 3ms/step - loss: nan - accuracy: 0.5310
